In [12]:
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.messages import HumanMessage
import typing

In [ ]:
import os

os.environ['GROQ_API_KEY'] = "entergroqapikey"
os.environ['TAVILY_API_KEY'] = "entertavilyapikey"

In [4]:
memory = MemorySaver()
model = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0)
Tavily_search = TavilySearch(max_results=5)
DuckDuckGo_search = DuckDuckGoSearchRun(max_results=5)
Wikipedia_search = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=5))

In [13]:
class WebSearchAgent:
    def __init__(self, tavily_search, duckduckgo_search, wikipedia_search, model, memory):
        self.tavily_search = tavily_search
        self.duckduckgo_search = duckduckgo_search
        self.wikipedia_search = wikipedia_search
        self.model = model
        self.memory = memory

    def search_with_fallback(self, query: str) -> typing.Dict[str, typing.Any]:
        tavily_result = self.try_tavily_search(query)
        if tavily_result and "response" in tavily_result:
            return {
                "success": True,
                "response": tavily_result["response"],
                "search_engine_used": "Tavily"
            }
        print("Tavily search failed, falling back to DuckDuckGo.")

        ddg_result = self.try_duckduckgo_search(query)
        if ddg_result and "response" in ddg_result:
            return {
                "success": True,
                "response": ddg_result["response"],
                "search_engine_used": "DuckDuckGo"
            }
        print("DuckDuckGo search failed, falling back to Wikipedia.")

        wiki_result = self.try_wikipedia_search(query)
        if wiki_result and "response" in wiki_result:
            return {
                "success": True,
                "response": wiki_result["response"],
                "search_engine_used": "Wikipedia"
            }
        
        print("All searches failed.")
        return {
            "success": False,
            "response": None,
            "search_engine_used": "none",
            "error": "All search engines failed"
        }
    
    def try_wikipedia_search(self, query: str) -> typing.Dict[str, typing.Any]:
        try:
            result = self.wikipedia_search.run(query)
            return {
                "response": result,
                "search_engine_used": "Wikipedia"
            }
        except Exception as e:
            print(f"Wikipedia search error: {e}")
            return {"error": str(e)}

    def try_tavily_search(self, query: str) -> typing.Dict[str, typing.Any]:
        try:
            result = self.tavily_search.run(query)
            return {
                "response": result,
                "search_engine_used": "Tavily"
            }
        except Exception as e:
            print(f"Tavily search error: {e}")
            return {"error": str(e)}

    def try_duckduckgo_search(self, query: str) -> typing.Dict[str, typing.Any]:
        try:
            result = self.duckduckgo_search.run(query)
            return {
                "response": result,
                "search_engine_used": "DuckDuckGo"
            }
        except Exception as e:
            print(f"DuckDuckGo search error: {e}")
            return {"error": str(e)}

    def query(self, query: str, thread_id: str = "default") -> typing.Dict[str, typing.Any]:
        try:
            print(f"Processing query: {query}")
            result = self.search_with_fallback(query)
            if not result["success"]:
                return {
                    "success": False,
                    "response": "I'm sorry, I couldn't retrieve search results due to technical issues. Please try again later.",
                    "error": result["error"],
                }
            config = {"configurable": {"thread_id": thread_id}}
            response = self.model.invoke([HumanMessage(content=f"Based on this search result: {result['response']}\n\nAnswer this question: {query}")])
            return {
                "success": True,
                "response": response.content,
                "search_engine_used": result["search_engine_used"]
            }
        except Exception as e:
            print(f"Error occurred in query: {e}")
            return {"error": str(e)}

In [14]:
web_agent = WebSearchAgent(Tavily_search, DuckDuckGo_search, Wikipedia_search, model, memory)
print("WebSearchAgent initialized successfully!")

WebSearchAgent initialized successfully!


In [15]:
test_result = web_agent.query("What is the latest news about AI?")
print(f"Test query result: {test_result['response']}")

Processing query: What is the latest news about AI?
Test query result: Based on the search results, here's a summary of the latest news about AI:

There are several recent developments in the field of AI. Some of the key news stories include:

1. **Microsoft's ChatGPT Update**: OpenAI is updating ChatGPT to better support users in mental distress. This comes after reports that people use the bot during emotional crises.
2. **AI Boosts Early Detection of Diabetic Eye Disease**: New research shows that AI can accurately screen for diabetic retinopathy, a leading cause of vision loss.
3. **Marketing AI Boom Faces Crisis of Consumer Trust**: There is a growing concern that the marketing AI boom may be facing a crisis of consumer trust.
4. **AI Security Wars: Can Google Cloud Defend?**: There is a growing concern about the security of AI systems, with Google Cloud facing questions about its ability to defend against AI security threats.

These are just a few examples of the latest news abou